# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [15]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des femmes journalistes belges

In [30]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P21 = gender
# Q6581072 = female
# P106 = Opccupation
# Q1930187 = Journalist
# P569 = date of birth
# P570 = date of death
# P1412 = language

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath ?lang WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P21 wd:Q6581072.
    ?person wdt:P106 wd:Q1930187 .
    ?person wdt:P1412 ?lang .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Women Belgian journalists found\n")
print(rows[:10])


63 Women Belgian journalists found

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q109768221'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Aliénor Debrocq'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1983-01-01T00:00:00Z'}, 'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q150'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66360955'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'An Brouckmans'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1958-03-24T00:00:00Z'}, 'lang': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7411'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q38554458'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'An Goovaerts'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': '

## Filtrer pour n'afficher que les noms des journalistes parlant Français

In [38]:
language_filter = 'Q150'
max_results = 20

date_format = "%Y-%m-%dT%H:%M:%SZ"
filtered_rows = [row for row in rows if language_filter in row['lang']['value']]
print(f"Les journalistes parlant français, {max_results} résultats max :\n")
for row in filtered_rows[:max_results]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = "????"
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = "????"
    except KeyError: # still alive
        death_year = ""
    print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

Les journalistes parlant français, 20 résultats max :

Aliénor Debrocq (1983-)
Anne Defraiteur Nicoleau (1972-)
Ariane Le Fort (1960-)
Béatrice Delvaux (1960-)
Caroline Boussart (1808-1891)
Charline Vanhoenacker (1977-)
Christine Aventin (1971-)
Christine Ockrent (1944-)
Colette Braeckman (1946-)
Dominique Costermans (1962-)
Elisa Brune (1966-2018)
Fatoumata Sidibé (1963-)
Hadja Lahbib (1970-)
Huguette de Broqueville (1931-2015)
Isabelle Spaak (1960-)
Jeanne Cappe (1895-1956)
Johanna Spaey (1966-)
Louise-Marie Danhaive (1923-1978)
Marie-Pascale Osterrieth (1956-)
Marie-Pascale Osterrieth (1956-)


## Pour en savoir plus

- [Le projet Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)
- [Aide à la construction de requêtes](https://query.wikidata.org/)